In [1]:
cd ..

/home/ayush/Documents/AI/Projects/Fake-news-detector


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    model_path :  Path
    tranformed_data: Path
    tokenizer_path : Path
    model_ckpt: str

In [3]:
from src.constants import *
from src.utils.common import read_yaml_file,create_dir

In [4]:
class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,
                 param=PARAM_FILE_PATH):
        self.config=read_yaml_file(config)
        self.param=read_yaml_file(param)
        create_dir([self.config.root_Artifact])

    def getmodeltrainer(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        create_dir([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            tranformed_data = Path(config.tranformed_data),
            tokenizer_path=Path(config.tokenizer_path),
            model_ckpt=config.model_ckpt
        )
        return model_trainer_config

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# tokenizer = AutoTokenizer.from_pretrained("therealcyberlord/fake-news-classification-distilbert")
# model = AutoModelForSequenceClassification.from_pretrained("therealcyberlord/fake-news-classification-distilbert")

/home/ayush/Documents/AI/Projects/Fake-news-detector/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments

[2025-06-12 07:45:02,217 : INFO : config : PyTorch version 2.7.1+cpu available.]
[2025-06-12 07:45:02,220 : INFO : config : TensorFlow version 2.19.0 available.]


2025-06-12 07:45:05.880326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 07:45:05.901280: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 07:45:06.064475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-12 07:45:06.201912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749694506.341324   21960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749694506.38

In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import pandas as pd


class ModelTrainer:
    def __init__(self, config):
        self.config = config

        # Initialize tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.config.model_ckpt, num_labels=2)

        self.df = pd.read_csv(self.config.tranformed_data) 
        print(self.df.columns)
        self.df = self.df.dropna(subset=["text"])
        self.df["text"] = self.df["text"].astype(str)
        train_df, test_df = train_test_split(
            self.df, test_size=0.2, stratify=self.df['label'], random_state=42
        )

        self.train_dataset = Dataset.from_pandas(train_df)
        self.test_dataset = Dataset.from_pandas(test_df)

        self.train_dataset = self.train_dataset.map(self.tokenize, batched=True)
        self.test_dataset = self.test_dataset.map(self.tokenize, batched=True)

    def tokenize(self, batch):
        return self.tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)

    def trainer(self):
        args = TrainingArguments(
            output_dir="./fine_tuned_roberta",
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            report_to="none",
            num_train_epochs=2,
            logging_steps=50,
        )

        trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=self.train_dataset,
            eval_dataset=self.test_dataset,
        )
        trainer.train()

        self.model.save_pretrained(self.config.model_path)
        self.tokenizer.save_pretrained(self.config.tokenizer_path)


In [12]:
config=ConfigurationManager()
getmodelTrainer = config.getmodeltrainer()
modeltrainer = ModelTrainer(config=getmodelTrainer)
# modeltrainer.tokenize()
# modeltrainer.trainer()


[2025-06-12 07:48:50,083 : INFO : common : Created directory at: artifact]
[2025-06-12 07:48:50,084 : INFO : common : Created directory at: artifact/model_trainer]
Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')


: 